In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        'input': 'What emojis represent "The Godfather"?',
        'output': """👨‍👨‍👦🔫🍝"""
    },
    {
        'input': 'What emojis represent "Top Gun"?',
        'output': """🛩️👨‍✈️🔥"""

    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "What emojis represent {input}"),
        ("human", "{output}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# def load_memory(_):
#     return memory.load_memory_variables({})["history"]

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Please reply with an 3 emoji."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "What do you know about {input}."),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(question):
    result = chain.invoke({"input": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


In [8]:
invoke_chain("Avater")

🌍🔥💧content='🌍🔥💧'


In [9]:
invoke_chain("Intern")

🌐💻📱content='🌐💻📱'


In [10]:
invoke_chain("First, ask them to tell you what movie they are asking about.")

🤔🎥🤷‍♂️content='🤔🎥🤷\u200d♂️'
